# Text extraction on image
Inga Ulusoy, SSC, July 2022

In [1]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    !pip install setuptools==61 -qqq
    # install ammico
    !pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [2]:
import ammico
from ammico import utils as mutils
from ammico import display as mdisplay

In [3]:
# download the models if they are not there yet
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.8 MB ? eta -:--:--

     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/42.8 MB 30.4 MB/s eta 0:00:02

     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/42.8 MB 82.6 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 21.0/42.8 MB 174.9 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 27.3/42.8 MB 181.7 MB/s eta 0:00:01

...

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/runner/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[nltk_data] Downloading package conll2000 to /home/runner/nltk_data...


[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/runner/nltk_data...


[nltk_data]   Unzipping corpora/movie_reviews.zip.


Finished.


In [4]:
images = mutils.find_files(path="data", limit=10)

In [5]:
mydict = mutils.initialize_dict(images)

## google cloud vision API
First 1000 images per month are free.

## Inspect the elements per image

In [6]:
mdisplay.explore_analysis(mydict, identify="text-on-image")

AttributeError: module 'ammico.display' has no attribute 'explore_analysis'

## Or directly analyze for further processing

In [7]:
for key in mydict:
    mydict[key] = ammico.text.TextDetector(
        mydict[key], analyse_text=True
    ).analyse_image()

400 DEATHS GET E-BOOK X AN Corporation ncy Services A municipal worker sprays disinfectant on his colleague after they cremated the body of a man who died due to COVID-19 at a crematorium in Ahmedabad on April 12, 2020. | Photo Credit: REUTERS


CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BEYOND THIS POINT 2019-nCov Coronavirus


NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW COUNTING METHOD RJ CITY HALL EXCLUDES 1,177 DEAD FROM COVID-19 STATISTICS G1 ARE TARGET OF PF OPERATION AGAINST FAKE NEWS OPERATION IS SEEN ON THE PLANALTO AT 10:09


## Convert to dataframe and write csv

In [8]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

In [9]:
# check the dataframe
df.head(10)

,filename,text,text_language,text_english,text_clean,text_english_correct,polarity,subjectivity,text_summary,sentiment,sentiment_score,entity,entity_type
0,data/102730_eng.png,400 DEATHS GET E-BOOK X AN Corporation ncy Ser...,en,400 DEATHS GET E-BOOK X AN Corporation ncy Ser...,DEATHS GET E - BOOK X AN Corporation Services ...,400 DEATHS GET E-BOOK X of Corporation ney Ser...,-0.125000,0.375000,A municipal worker sprays disinfectant on his...,NEGATIVE,0.991692,"[AN Corporation ncy Services, Ahmedabad, RE, #...","[ORG, LOC, PER, ORG]"
1,data/102141_2_eng.png,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,en,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,0.000000,0.000000,Coronavirus QUARANTINE CORONAVIRUS OUTBREAK,NEGATIVE,0.976247,"[CORONAVIRUS, ##AR, ##TI, ##RONAVIR, ##C, Co]","[ORG, MISC, MISC, ORG, MISC, MISC]"
2,data/106349S_por.png,NEWS URGENTE SAMSUNG AO VIVO Rio de Janeiro NO...,pt,NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW CO...,NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW CO...,NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW CO...,-0.106818,0.588636,"NEW COUNTING METHOD RJ City HALL EXCLUDES 1,1...",NEGATIVE,0.990659,"[Rio de Janeiro, C, ##IT, P, ##NA, ##LTO]","[LOC, ORG, LOC, LOC, ORG, LOC]"


In [10]:
# Write the csv
df.to_csv("./data_out.csv")